In [ ]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), 'detection_copied_from_timewak'))
sys.path.append(os.path.abspath('.')) # to run files that are away
os.environ["WANDB_SILENT"] = "true"  # Suppress WandB logs

libraries = ["torch", "numpy", "pandas"]
modules   = {lib: sys.modules.get(lib) for lib in libraries}

if not modules["torch"]:
    import torch
    # from torch import tensor, zeros, isnan, abs, fft, std, float32 as torch_float32
if not modules["numpy"]:
    import numpy as np
if not modules["pandas"]:
    import pandas as pd


In [ ]:
"""Encoder"""

import torch.nn as nn

# 1. Data
x = np.random([100, 100])

# 2. Encoder

# 3. Decoder

# 4. Loss
mae_loss = (1/n) * np.sum((x-x)^2)

# 5. Optimization

for i in x:
    loop
